In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t

/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1170: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


_ColormakerRegistry()

In [3]:
molsys = m3t.load('/home/diego/1brs.pdb')

si


TypeError: from_pdb() got an unexpected keyword argument 'selection'

In [ ]:
import numpy as np

In [ ]:
np.arange(0,dtype='int64')

In [ ]:
range(0)

In [ ]:
from molmodmt.multitool import _dict_get